# Integrating DB2 on Docker with Jupyter Notebooks

This notebook creates a Docker container running the latest DB2 image and executes some simple queries before removing the container.

See the associated blog entry at: https://www.triton.co.uk/integrating-db2-on-docker-with-jupyter-notebooks/

In [ ]:
%pip install docker --user
%pip install ibm_db --user
%pip install ibm_db_sa --user
%pip install ipython-sql --user
%pip install matplotlib --user

In [ ]:
import os
os._exit(00)

In [ ]:
dockerImageName='JupyterDB2'
dockerStoragePath='/home/docker_files/volumes'

databaseInstancePassword='testpass'
databaseInstanceExternalPort='50066'

In [ ]:
import docker
import ibm_db
import time
import matplotlib

%reload_ext sql

In [ ]:
cli = docker.from_env()

In [ ]:
cli.images.pull('ibmcom/db2:latest')

In [ ]:
cli.containers.run(
    'ibmcom/db2:latest',
    auto_remove=True,
    privileged=True,
    detach=True,
    name=dockerImageName,
    environment={"LICENSE":"accept","DB2INST1_PASSWORD":databaseInstancePassword,"SAMPLEDB":"true"},
    ports={'50000': databaseInstanceExternalPort},
    volumes={dockerStoragePath + dockerImageName: {'bind': '/database', 'mode': 'rw'}}
)

In [ ]:
container=(_)
fin=0

while fin == 0:
    if (container.logs(tail=15).find(b'Setup has completed.') != -1):
        fin=1
    else:
        print('Waiting for container to finish starting: ' + str(time.asctime(time.localtime())))
        time.sleep(10)
        
print('DB container ready!')

In [ ]:
%sql db2+ibm_db://db2inst1:$databaseInstancePassword@localhost:$databaseInstanceExternalPort/sample

In [ ]:
%%sql SELECT 'Hello world! In container ' CONCAT host_name CONCAT ' the time is: ' 
CONCAT CHAR(current_timestamp) FROM TABLE(sysproc.env_get_sys_info()) as systeminfo;

In [ ]:
chart = %sql select year(hiredate) as hire_year, count(*) as num_employees from employee group by year(hiredate)
chart.bar();

In [ ]:
container.stop()